In [136]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_before)]
    after_season =  transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(before_season.iloc[0]['Market Value']) * .8), "Only Before"
                else:
                    return (float(before_season.iloc[0]['Market Value']) * 1.2), "Only Before"
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(after_season.iloc[0]['Market Value']) * .8), "Only After"
                else:
                    return (float(after_season.iloc[0]['Market Value']) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        if before_season_val != "-" and after_season_val != "-":
            return ((float(before_season.iloc[0]['Market Value']) + float(after_season.iloc[0]['Market Value'])) / 2), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(before_season.iloc[0]['Market Value']) * .8), "Used Season Before * .8 But Had Both"
            else:
                return (float(before_season.iloc[0]['Market Value']) * 1.2), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(after_season.iloc[0]['Market Value']) * .8), "Used Season After * .8 But Had Both"
            else:
                return (float(after_season.iloc[0]['Market Value']) * 1.2), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, transfermarkt_data):
    row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == season) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_up_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)+1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_down_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)-1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    #print(len(row), len(one_down_row), len(one_up_row))
    if not row.empty:
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if marketval != 0:
            return marketval, "Found in Curr Season"
        else:
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"

def last_name_match(name, dataset_of_names):
    matching_names = []
    for full_name in dataset_of_names:
        
        # Split the full name into tokens
        full_name_tokens = set(re.split(r'\s+|[-–]', full_name))
        
        # Check if any token matches the last name or its unidecoded version
        for token in full_name_tokens:
            #if full_name == 'Fródi Benjaminsen':
                #print(token, name, token == name)
                #print(full_name_tokens)
            
            if token == name or unidecode(token) == name:
                matching_names.append(full_name)
                break  # Once a match is found, move to the next name
    return matching_names

def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    
    return possible_names[0]

def add_correct_money(money_column):

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0
    

def remove_nation(match, nation1):
    # Split the match string by 'vs' and strip any leading or trailing spaces
    parts = [part.strip() for part in match.split('vs')]
    # Remove the nation1 from the list of parts
    parts.remove(nation1)
    # Join the remaining parts and return
    return ''.join(parts)

In [2]:
#leagues value
#leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
transfermarkt_data = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/MarketValuesComplete.csv')

#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

results_df = load_csv_dataset('newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_900/4099345605.py:49: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [22]:
large_combined_df = load_csv_dataset('combined_DF_merged_April_24.csv')
large_combined_df = large_combined_df.drop(columns={'Unnamed: 0'})

#filter large combined DF 
large_combined_df_working = large_combined_df[(large_combined_df['Lookup Return Case'] == 'working') | 
                                              (large_combined_df['Lookup Return Case'] == 'Found in Curr Season') | 
                                              (large_combined_df['Lookup Return Case'] == 'Used Large Dataset')]

large_combined_df_not_working = large_combined_df[(large_combined_df['Lookup Return Case'] != 'working') &
                                              (large_combined_df['Lookup Return Case'] != 'Found in Curr Season') &
                                              (large_combined_df['Lookup Return Case'] != 'Used Large Dataset')].reset_index()

large_combined_df_not_working['Status New'] = ''
large_combined_df_not_working['Market Value New'] = 0

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_900/4099345605.py:49: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [ ]:
#Cases we want to start with 
    #Did Not Find in Curr Season, One Up, or One Down
    #Both Empty 

#plan

    #start with the cases above


In [195]:
large_combined_df_not_working[(number_row + 20):(number_row + 40)] #10:20

,index,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status New,Market Value New
290,62603,1103,"Euros 2024, Qualifiers",Liechtenstein vs Portugal,16.11.23,Portugal,PT,2023,SUCCESS,Tiago Gomes,0,False,"Did Not Find in Curr Season, One Up, or One Down",False,Tiago Gomes,Tote Gomes,single,,0
291,62945,3137,"World Cup Qualifiers, Europe, 2018",Switzerland vs Faroe Islands,13-11-2016,Faroe Islands,FO,2016,SUCCESS,Sölvi Vatnhamar,0,False,Had Both but Both Were Empty,False,Sölvi Vatnhamar,S Vatnhamar,single,,0
292,62946,3147,"World Cup Qualifiers, Europe, 2018",San Marino vs Czech Republic,26-03-2017,San Marino,SM,2017,SUCCESS,Davide Cesarini,0,False,Had Both but Both Were Empty,False,Davide Cesarini,D Cesarini,single,,0
293,62947,3147,"World Cup Qualifiers, Europe, 2018",San Marino vs Czech Republic,26-03-2017,San Marino,SM,2017,SUCCESS,Danilo Rinaldi,0,False,Had Both but Both Were Empty,False,Danilo Rinaldi,D Rinaldi,single,,0
294,62948,3147,"World Cup Qualifiers, Europe, 2018",San Marino vs Czech Republic,26-03-2017,San Marino,SM,2017,SUCCESS,Matteo Vitaioli,0,False,Had Both but Both Were Empty,False,Matteo Vitaioli,M Vitaioli,single,,0
295,62949,3151,"World Cup Qualifiers, Europe, 2018",Gibraltar vs Greece,06-09-2016,Gibraltar,GI,2016,SUCCESS,Ryan Casciaro,0,False,Had Both but Both Were Empty,False,Ryan Casciaro,R Casciaro,single,,0
296,62950,3151,"World Cup Qualifiers, Europe, 2018",Gibraltar vs Greece,06-09-2016,Gibraltar,GI,2016,SUCCESS,Jean-Carlos Garcia,0,False,Had Both but Both Were Empty,False,Jean-Carlos Garcia,J Garcia,single,,0
297,62951,3151,"World Cup Qualifiers, Europe, 2018",Gibraltar vs Greece,06-09-2016,Gibraltar,GI,2016,SUCCESS,Jamie Bosio,0,False,Had Both but Both Were Empty,False,Jamie Bosio,J Bosio,single,,0
298,62952,3151,"World Cup Qualifiers, Europe, 2018",Gibraltar vs Greece,06-09-2016,Gibraltar,GI,2016,SUCCESS,Anthony Hernandez,0,False,Had Both but Both Were Empty,False,Anthony Hernandez,Anthony Hernandez,single,,0
299,62953,3161,"World Cup Qualifiers, Europe, 2018",Portugal vs Andorra,07-10-2016,Andorra,AD,2016,SUCCESS,Josep Gomes,0,False,Had Both but Both Were Empty,False,Josep Gomes,Josep Gómes,single,,0


In [246]:
done_cases = 0
success_cases = 0
not_done_cases = 0
number_row = 350
for i in range(4000, 9000): #number_row, (number_row + 10)  #len(large_combined_df_not_working)
    row = large_combined_df_not_working.loc[i]
    if row['Status New'] != 'Done' and row['Status New'] != 'Multiple' and row['Status New'] != 'OLYMPICS' and row['Status New'] != 'U-20' and ((row['Lookup Return Case'] == 'Both Empty') | (row['Lookup Return Case'] == 'Did Not Find in Curr Season, One Up, or One Down')):

        jersey = row['ORIGINAL JERSEY']
        names_found = row['Name(s) Found']
        last_name = re.split(r'\s+|[-–]', jersey)[-1]
        correct_season = row['Season']
        this_country_code = row['Team Country Code']
        
        test = check_other_team(row, transfermarkt_data, results_df)
        if test == ['No matches found even with the lowest threshold.']: #checking the OTHER NATION left no potential matches            
            dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == this_country_code]['Name'].unique()

            #TRY TO FIND OTHER PLAYERS FROM THE NATION W THE SAME LAST NAME
            names_same_nation_last_name = last_name_match(last_name, dataset_nationality)
            if names_same_nation_last_name == None:
                print(i, 'no matches in other country, and last name match returned nobody')
                not_done_cases += 1
            else:
                #SOME PLAYERS HAVE THE SAME LAST NAME AS THE ROW
                for name_candidate in names_same_nation_last_name:
                    num_seasons_around = len(transfermarkt_data[(transfermarkt_data['Name'] == name_candidate) & (transfermarkt_data['Season'].isin([correct_season, correct_season + 1, correct_season - 1]))])
                    if num_seasons_around == 0:
                        names_same_nation_last_name.remove(name_candidate)

                #IF MORE THAN 2 PLAYERS HAVE THE SAME LAST NAME
                if len(names_same_nation_last_name) >= 2:
    
                    correct_names = AssemblyHelpers.multiNameMatchDateLookup(names_same_nation_last_name, row['Nationality'], row['Date']) 
                    if type(correct_names) == str:
                        correct_names = [correct_names]
                    if len(correct_names) == 1:
                        correct_name = correct_names[0]
                        number, reason = find_in_transfermarkt(correct_name, correct_season, this_country_code, transfermarkt_data)
                        print(f'{i} *** single return. correct name is {correct_name}. jersey is {jersey}', number, reason)

                        #WORKING FOR THE GUY WE FOUND
                        if reason == 'Found in Curr Season':
                            if number == '-':
                                print(i, 'this guy has no market value')
                                #number = 0
                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                            else:
                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'working'
                            large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                            large_combined_df_not_working.at[i, 'Name(s) Found'] = correct_name
                            large_combined_df_not_working.at[i, 'Name'] = correct_name
                            large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                            success_cases += 1
                        else: #THE GUY WE FOUND HAD NO DATA IN THE SEASON WE WANTED
                            
                            if reason == 'Did Not Find in Curr Season, One Up, or One Down':
                                #TRY TO FIND THIS PLAYER IN THE LARGER DATASET 
                                number, reason = find_in_transfermarkt(correct_name, correct_season, correct_country_code, leagues_value_large)
                                if reason == 'Found in Curr Season':
                                    if number == '-':
                                        print(i, 'used large dataset to find guy. this guy has no market value')
                                        #number = 0
                                        large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                    else:
                                        large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Used Large Dataset'
                                        print(i, 'used large dataset to find guy')
                                    large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                    large_combined_df_not_working.at[i, 'Name(s) Found'] = correct_name
                                    large_combined_df_not_working.at[i, 'Name'] = correct_name
                                    large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                    success_cases += 1
                                else:
                                    #FAILED IN THE LARGER DATASET 
                                    print(i, 'FOUND THE GUY IN THE OTHER NATION. tried using larger dataset after main one had no data. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                    not_done_cases += 1
                            else:
                                #FAILED IN THE MAIN DATASET 
                                print(i, 'FOUND THE GUY IN THE OTHER NATION. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                    else:
                        print(i, f'multiple guys w the same last name from {row['Nationality']} match date filter left multiple: ', correct_names)
                        success_cases += 1
                        large_combined_df_not_working.at[i, 'Name(s) Found'] = correct_names
                        large_combined_df_not_working.at[i, 'Status New'] = 'Multiple'
                
                #IF ONLY 1 PLAYER HAS THE SAME LAST NAME
                elif len(names_same_nation_last_name) == 1:
                    
                    #ADD $ INFO FOR SINGLE MATCHING LAST NAME GUY
                    correct_name = names_same_nation_last_name[0]
                    number, reason = find_in_transfermarkt(correct_name, correct_season, row['Team Country Code'], transfermarkt_data)
                    print(f'{i} *** single return. correct name is {correct_name}. jersey is {jersey}', number, reason)
                    if reason == 'Found in Curr Season':
                        if number == '-':
                            print(i, 'this guy has no market value')
                            #number = 0
                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                        else:
                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'working'
                        large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                        large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                        success_cases += 1
                    else:
                        print(i, correct_name, 'tried to find money info but it did not work. reason: ')
                        not_done_cases += 1
                else:
                    #Nobody with this last name WITH CURRENT or +/- 1 seasons from the nation
                    print(i, 'Nobody with this last name WITH CURRENT or +/- 1 seasons from the nation', names_same_nation_last_name, row['ORIGINAL JERSEY'])
                    #TRYING TO FIND PLAYERS USING THE LARGER DATASET
                    
                    dataset_nationality_large = leagues_value_large[leagues_value_large['Team 1 Code'] == this_country_code]['Name'].unique()
                    names_same_nation_last_name_large_dataset = last_name_match(last_name, dataset_nationality_large)    
                    if names_same_nation_last_name_large_dataset == None:
                        print(i, 'last name match using large dataset ALSO returned nobody')
                        not_done_cases += 1
                    else:
                        print(i, 'last name match using large dataset returned ', names_same_nation_last_name_large_dataset)
                        not_done_cases += 1
                            ###FINISH THIS
        else: #checking the other nation left potential matches
            if len(test) >= 1:
                correct_names = []
                candidates = [names_found]
                for test_name in test:
                    candidates.append(test_name)

                #KEEP ALL THE MATCHES FROM EITHER NATION WITH THE SAME LAST NAME 
                for name_candidate in candidates:
                    name_tokens = re.split(r'\s+|[-–]', name_candidate)
                    for token in name_tokens:
                        if token == last_name:
                            correct_names.append(name_candidate)
                            break
                if len(correct_names) >= 2: #IF MORE THAN 2 PLAYERS HAVE THE SAME LAST NAME (FROM BOTH NATIONS COMBINED)
                    print(i, 'match date filter required here for multiple nations', correct_names)

                    #SPLIT NAMES INTO LIST FOR EACH NATION
                    nation1 = row['Nationality']
                    nation2 = remove_nation(row['Match'], nation1)
                    names_nation_1 = []
                    names_nation_2 = []
                    for name in correct_names:
                        if len(transfermarkt_data[(transfermarkt_data['Name'] == name) & (transfermarkt_data['Nationality'] == nation1)]) >= 1:
                            names_nation_1.append(name)
                        elif len(transfermarkt_data[(transfermarkt_data['Name'] == name) & (transfermarkt_data['Nationality'] == nation2)]) >= 1:
                            names_nation_2.append(name)
                    #for each nation, filter the names Found using teh match date
                    correct_names_nation_1 = AssemblyHelpers.multiNameMatchDateLookup(names_nation_1, nation1, row['Date']) 
                    correct_names_nation_2 = AssemblyHelpers.multiNameMatchDateLookup(names_nation_2, nation2, row['Date']) 

                    #COMBINING THE RESULTING FILTERED LISTS FROM EACH NATION
                    if type(correct_names_nation_2) == str:
                        correct_names_nation_2 = [correct_names_nation_2]
                    if type(correct_names_nation_1) == str:
                        correct_names_nation_1 = [correct_names_nation_1]
                    if correct_names_nation_1 == []:
                        set_of_names = correct_names_nation_2
                    elif correct_names_nation_2 == []:
                        set_of_names = correct_names_nation_1
                    else:
                        set_of_names = list(set(correct_names_nation_1 + correct_names_nation_2))
                    #IF THE RESULT IS ONLY 1 PLAYER
                    if len(set_of_names) == 1:
                        correct_name = set_of_names[0]
                        #IF SINGLE PLAYER IS FROM NATION 1, FIND HIS $
                        if correct_name in names_nation_1:
                            print(i, 'OTHER nation matching left 1 result', correct_name, nation1)
                            correct_country = nation1
                            correct_country_code = countries_codes[countries_codes['Country'] == nation1][' Code'].unique()[0]
                            number, reason = find_in_transfermarkt(correct_name, correct_season, correct_country_code, transfermarkt_data)
                        #IF SINGLE PLAYER IS FROM NATION 2, FIND HIS $
                        elif correct_name in names_nation_2:
                            print(i, 'OTHER nation matching left 1 result', correct_name, nation2)
                            correct_country = nation2
                            correct_country_code = countries_codes[countries_codes['Country'] == nation2][' Code'].unique()[0]
                            number, reason = find_in_transfermarkt(correct_name, correct_season, correct_country_code, transfermarkt_data)
                        
                        print(f'{i} *** FOUND THE GUY IN THE OTHER NATION. single return. correct name is {correct_name}. jersey is {jersey}', number, reason)
                        #IF FINDING $ INFO FOR THAT GUY WORKS, ADD TO THE DATASET
                        if reason == 'Found in Curr Season':
                            if number == '-':
                                print(i, 'this guy has no market value')
                                #number = 0
                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                            else:
                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'working'
                            large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                            large_combined_df_not_working.at[i, 'Nationality'] = correct_country
                            large_combined_df_not_working.at[i, 'Team Country Code'] = correct_country_code
                            large_combined_df_not_working.at[i, 'Name(s) Found'] = correct_name
                            large_combined_df_not_working.at[i, 'Name'] = correct_name
                            large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                            success_cases += 1
                        else: #THE GUY WE FOUND HAD NO DATA IN THE SEASON WE WANTED
                            
                            if reason == 'Did Not Find in Curr Season, One Up, or One Down':
                                #TRY TO FIND THIS PLAYER IN THE LARGER DATASET 
                                number, reason = find_in_transfermarkt(correct_name, correct_season, correct_country_code, leagues_value_large)
                                if reason == 'Found in Curr Season':
                                    if number == '-':
                                        print(i, 'used large dataset to find guy. this guy has no market value')
                                        #number = 0
                                        large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                    else:
                                        large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Used Large Dataset'
                                        print(i, 'used large dataset to find guy')
                                    large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                    large_combined_df_not_working.at[i, 'Nationality'] = correct_country
                                    large_combined_df_not_working.at[i, 'Team Country Code'] = correct_country_code
                                    large_combined_df_not_working.at[i, 'Name(s) Found'] = correct_name
                                    large_combined_df_not_working.at[i, 'Name'] = correct_name
                                    large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                    success_cases += 1
                                else:
                                    #FAILED IN THE LARGER DATASET 
                                    print(i, 'FOUND THE GUY IN THE OTHER NATION. tried using larger dataset after main one had no data. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                    not_done_cases += 1
                            else:
                                #FAILED IN THE MAIN DATASET 
                                print(i, 'FOUND THE GUY IN THE OTHER NATION. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                    elif len(set_of_names) >= 2:
                        print(i, 'Multiple people match this name from both nations, and they seemingly both played. ', set_of_names, jersey)
                        not_done_cases += 1
                        #TALK TO TERRELL ABOUT WHAT TO DO HERE. YOU NEED TO FIND THE OTHER ROW WHERE THE GUY WOULD BE...
                    else:
                        print(i, f"jersey is {jersey}, match is {row['Match']}, date is {row['Date']}. list from {nation1} was {names_nation_1}. list from {nation2} is {names_nation_2}")

                    #not_done_cases += 1
                elif len(correct_names) == 1: #IF 1 PLAYER HAS THE SAME LAST NAME (FROM BOTH NATIONS COMBINED)
                    print(i, f"game was {row['Match']}, nation is {row['Nationality']}")
                    correct_season = row['Season']
                    correct_name = correct_names[0]
                    number, reason = find_in_transfermarkt(correct_name, correct_season, row['Team Country Code'], transfermarkt_data)
                    print(f'{i} *** single return. correct name is {correct_name}. jersey is {jersey}', number, reason)
                    if reason == 'Found in Curr Season':
                        if number == '-':
                            print(i, 'this guy has no market value')
                            #number = 0
                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                        else:
                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'working'
                        large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                        large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                        success_cases += 1
                    else:
                        print(i, 'lookup gave diff reason. reason was ', reason, correct_name, jersey)

                        if reason == 'Did Not Find in Curr Season, One Up, or One Down':
                            number, reason = find_in_transfermarkt(correct_name, correct_season, row['Team Country Code'], leagues_value_large)

                            if reason == 'Found in Curr Season':
                                if number == '-':
                                    print(i, 'used large dataset to find guy. this guy has no market value')
                                    #number = 0
                                    large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                else:
                                    large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Used Large Dataset'
                                large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                success_cases += 1
                            else:
                                print(i, 'tried using larger dataset after main one had no data. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                not_done_cases += 1
                else: #nobody from the other nation or the original nation had the same last name. candidates are 
                    
                    #FINDING ALL PLAYERS W THE SAME LAST NAME AS THE JERSEY
                    names_same_nation_last_name = last_name_match(last_name, dataset_nationality)
                    if names_same_nation_last_name != []: #SOME LAST NAME MATCHES
                        
                        if len(names_same_nation_last_name) == 1:
                            correct_name = names_same_nation_last_name[0]
                            number, reason = find_in_transfermarkt(correct_name, correct_season, this_country_code, transfermarkt_data)
                            print(f'{i} *** single return. correct name is {correct_name}. jersey is {jersey}', number, reason)
                            if reason == 'Found in Curr Season':
                                if number == '-':
                                    print(i, 'this guy has no market value')
                                    #number = 0
                                    large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                success_cases += 1
                            else:
                                print(i, 'lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                if reason == 'Did Not Find in Curr Season, One Up, or One Down':
                                    number, reason = find_in_transfermarkt(correct_name, correct_season, this_country_code, leagues_value_large)

                                    if reason == 'Found in Curr Season':
                                        if number == '-':
                                            print(i, 'used large dataset to find guy. this guy has no market value')
                                            #number = 0
                                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                        else:
                                            large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Used Large Dataset'
                                        large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                        large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                        success_cases += 1
                                    else:
                                        print(i, 'tried using larger dataset after main one had no data. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                        not_done_cases += 1
                        else:
                            print(i, 'some last name matches', names_same_nation_last_name)
                            not_done_cases += 1
                                ###FINISH THIS
                    else: #ZERO LAST NAME MATCHES

                        #FIND THE CLOSEST MATCH USING THE ORIGINAL NAMES IT FOUND AND THE NAMES FOUND FROM THE OTHER NATION (test)
                        best_name_from_choices = threshold_player_match(jersey, candidates)[0]
                        
                        if len(best_name_from_choices) == 1: #IF THERE IS 1 BEST MATCH
                            if best_name_from_choices[0] == names_found: #IF THE BEST MATCH IT FINDS IS THE NAME THAT WAS ORIGINALLY IN THE ROW
                                correct_name = best_name_from_choices[0]
                                number, reason = find_in_transfermarkt(correct_name, correct_season, row['Team Country Code'], transfermarkt_data)
                                print(f'{i} *** single return. correct name is {correct_name}. jersey is {jersey}', number, reason)
                                if reason == 'Found in Curr Season':
                                    if number == '-':
                                        print(i, 'this guy has no market value')
                                        #number = 0
                                        large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                    large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                    large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                    success_cases += 1
                                else:
                                    print(i, 'lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                    if reason == 'Did Not Find in Curr Season, One Up, or One Down':
                                        number, reason = find_in_transfermarkt(correct_name, correct_season, row['Team Country Code'], leagues_value_large)

                                        if reason == 'Found in Curr Season':
                                            if number == '-':
                                                print(i, 'used large dataset to find guy. this guy has no market value')
                                                #number = 0
                                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Guy has 0 market value in this season'
                                            else:
                                                large_combined_df_not_working.at[i, 'Lookup Return Case'] = 'Used Large Dataset'
                                            large_combined_df_not_working.at[i, 'Status New'] = 'Done'
                                            large_combined_df_not_working.at[i, 'Market Value New'] = add_correct_money(number)
                                            success_cases += 1
                                        else:
                                            print(i, 'tried using larger dataset after main one had no data. lookup gave diff reason. reason was ', reason, correct_name, jersey)
                                            not_done_cases += 1
                            else: #IF THE BEST MATCH IT FINDS IS NOT THE NAME THAT WAS ORIGINALLY IN THE ROW
                                print(i, 'only one name found but it wasnt the one that we found before.', best_name_from_choices, names_found, jersey)
                                not_done_cases += 1
                        else:
                            print(i, 'choosing between threshold name and already foudn name left multiple people.', best_name_from_choices)
                            not_done_cases += 1
                                ###FINISH THIS - MATCH DATE
            else: #SOMEHOW TEST IS NOT AT LEAST LENGTH 1 BUT IT DOESNT SAY "no matches found even w lowest threshold"
                print(i, test, names_found, jersey)
    else: #DONE CASES
        print(i, '***DONE***')
        done_cases += 1

if done_cases == 20:
    print('All Done')
else:
    print(done_cases, success_cases, not_done_cases)

# for i in range(number_row, (number_row + 20)):
#     if large_combined_df_not_working.at[i, 'Name(s) Found'] == []:
#         large_combined_df_not_working.at[i, 'Status New'] = ''

    #MAKE SURE ITS A BAD EDGE CASES

#for each nation, filter with it 
    #make a list of each nation and the players in it

#later 
    #is there a need to, if you have a list of names from multiple nations, just check if each person has data in the season of the match  

4000 ***DONE***
4001 ***DONE***
4002 ***DONE***
4003 ***DONE***
4004 ***DONE***
4005 ***DONE***
4006 ***DONE***
4007 ***DONE***
4008 ***DONE***
4009 ***DONE***
4010 ***DONE***
4011 ***DONE***
4012 ***DONE***
4013 ***DONE***
4014 ***DONE***
4015 ***DONE***
4016 ***DONE***
4017 ***DONE***
4018 ***DONE***
4019 ***DONE***
4020 ***DONE***
4021 ***DONE***
4022 ***DONE***
4023 ***DONE***
4024 ***DONE***
4025 ***DONE***
4026 ***DONE***
4027 ***DONE***
4028 ***DONE***
4029 ***DONE***
4030 ***DONE***
4031 ***DONE***
4032 ***DONE***
4033 ***DONE***
4034 ***DONE***
4035 ***DONE***
4036 ***DONE***
4037 ***DONE***
4038 ***DONE***
4039 ***DONE***
4040 ***DONE***
4041 ***DONE***
4042 ***DONE***
4043 ***DONE***
4044 ***DONE***
4045 ***DONE***
4046 ***DONE***
4047 ***DONE***
4048 ***DONE***
4049 ***DONE***
4050 ***DONE***
4051 ***DONE***
4052 ***DONE***
4053 ***DONE***
4054 ***DONE***
4055 ***DONE***
4056 ***DONE***
4057 ***DONE***
4058 ***DONE***
4059 ***DONE***
4060 ***DONE***
4061 ***DONE***
4062 ***

In [332]:
len(large_combined_df)

83932

In [261]:
large_combined_df_not_working['Competition'].value_counts()

Competition
World Cup Qualifiers, Asia, 2018             1812
World Cup Qualifiers, Asia, 2022              953
World Cup Qualifiers, Africa, 2022            887
World Cup 2022, Qualifiers, CONCACAF          817
World Cup Qualifiers, Africa, 2018            717
Euros 2024, Qualifiers                        695
World Cup Qualifiers, Europe, 2022            686
Euros 2020, Qualifiers                        605
World Cup Qualifiers, Europe, 2018            526
World Cup 2018, Qualifiers, CONMEBOL          362
World Cup 2018, Qualifiers, CONCACAF          312
World Cup Qualifiers, Africa, 2026            300
Gold Cup, 2023                                299
Euros 2016, Qualifiers                        276
World Cup Qualifiers, Africa, 2014            213
World Cup 2022, Qualifiers, CONMEBOL          211
World Cup Qualifiers, Europe, 2014            204
Gold Cup, 2013                                183
World Cup 2014, Qualifiers, CONCACAF          161
World Cup Qualifiers, Asia, 2026      

In [202]:
for i in range(0, len(large_combined_df_not_working)):
    if large_combined_df_not_working.at[i, 'ORIGINAL JERSEY'] == 'Mc Burnette':
        #print(i)
        # large_combined_df_not_working.at[i, 'Name(s) Found'] = 'Mohamed Koffi' M Ko 
        # large_combined_df_not_working.at[i, 'Name'] = 'Mohamed Koffi'
        large_combined_df_not_working.at[i, 'ORIGINAL JERSEY'] = 'Nazir McBurnette'

        #352, 365 is Kuk-Chol Kang

315
326
341
349
1266
